In [2]:
import requests
from utils import  AGENCIES_LIST_URL
import json
import pandas as pd

from utils import  get_project_root

git_root = get_project_root()
data_dir = git_root / "data/"



AGENCIES_PATH = "/api/admin/v1/agencies.json"
r = requests.get(AGENCIES_LIST_URL)
agencies = r.json()['agencies']
agencies_filename = data_dir / 'agencies.json'
with open(agencies_filename, 'w') as f:
    json.dump(agencies, f, indent=4)
df = pd.read_json(agencies_filename)
# Check the structure of the cfr_references column to determine which section types can be used
all = df['cfr_references'].to_list()
types = set()
for l in all:
    if isinstance(l, list):
        for d in l:
            for k, v in d.items():
                types.add(k)
    else:
        for k, v in d.items():
            types.add(k)
types
XML_DOCUMENT_SCHEMA_MAP = {
    "chapter": {"element": "DIV3", "type": "CHAPTER"},
    "part": {"element": "DIV5", "type": "PART"},
    "subchapter": {"element": "DIV4", "type": "SUBCHAP"},
    "subtitle": {"element": "DIV2", "type": "SUBTITLE"},
}
import re

def word_count_sans_xml(xml_string):
    """
    Strip XML tags from a document using regex and count words.
    
    Args:
        xml_string (str): The XML content as a string
    
    Returns:
        str: Plain text with XML tags removed
        int: Word count
    """
    # Remove all XML tags
    text_only = re.sub(r'<[^>]+>', ' ', xml_string)
    
    # Clean up whitespace (multiple spaces, newlines, etc.)
    clean_text = re.sub(r'\s+', ' ', text_only).strip()
    
    # Count words
    word_count = len(clean_text.split())
    
    return word_count

word_count_sans_xml(s)
from lxml import etree
# Parse the XML

def get_word_count_for_agency(agency_references_json):
    word_count = 0
    for json_path in agency_references_json:
        title_number = json_path['title']
        with open(data_dir / f"raw_title_xml/title_{title_number}.xml", 'r') as f:
            xml_string = f.read()
        parser = etree.XMLParser(recover=True)  # More forgiving parser
        root = etree.fromstring(xml_string.encode('utf-8'), parser)
        for k, v in json_path.items():
            if k == 'title':
                continue
            element = XML_DOCUMENT_SCHEMA_MAP[k]['element']
            type = XML_DOCUMENT_SCHEMA_MAP[k]['type']
            # Find the part (DIV5)
            xpath = f".//{element}[@N='{v}']"
            elements = root.xpath(xpath)
            if len(elements) == 0:
                print(f"Element not found: {xpath}")
                continue
            if len(elements) > 1:
                print(f"Multiple elements found: {xpath}")
                continue
            element = elements[0]
            root = element
            
        # Convert the element back to string
        s = etree.tostring(root, encoding='unicode', pretty_print=True)
        word_count += word_count_sans_xml(s)
    return word_count
# Assuming this is pretty well formed data. Two agencies don't point to the same data, there is no orphaned data, etc.
child_dfs = []
for i, row in df.iterrows():
    agency = row['name']
    children_df = pd.DataFrame(row['children'])
    if len(children_df) > 0:
        children_df['parent_agency'] = agency
        child_dfs.append(children_df)

child_dfs = pd.concat(child_dfs, ignore_index=True)
df['parent_agency'] = df['name']
all_df = pd.concat([df, child_dfs], ignore_index=True)
all_df['word_count'] = all_df['cfr_references'].apply(get_word_count_for_agency)
all_df.to_csv(data_dir / 'agencies_word_count.csv', index=False)

In [44]:
all_df

,name,short_name,display_name,sortable_name,slug,children,cfr_references,parent_agency,word_count
0,Administrative Conference of the United States,ACUS,Administrative Conference of the United States,Administrative Conference of the United States,administrative-conference-of-the-united-states,[],"[{'title': 1, 'chapter': 'III'}]",Administrative Conference of the United States,12395
1,Advisory Council on Historic Preservation,ACHP,Advisory Council on Historic Preservation,Advisory Council on Historic Preservation,advisory-council-on-historic-preservation,[],"[{'title': 36, 'chapter': 'VIII'}]",Advisory Council on Historic Preservation,31039
2,Special Inspector General for Afghanistan Reco...,SIGAR,Special Inspector General for Afghanistan Reco...,"Afghanistan Reconstruction, Special Inspector ...",special-inspector-general-for-afghanistan-reco...,[],"[{'title': 5, 'chapter': 'LXXXIII'}]",Special Inspector General for Afghanistan Reco...,10987
3,African Development Foundation,USADF,African Development Foundation,African Development Foundation,african-development-foundation,[],"[{'title': 22, 'chapter': 'XV'}, {'title': 48,...",African Development Foundation,33526
4,United States Agency for Global Media,USAGM,United States Agency for Global Media,"Agency for Global Media, United States",united-states-agency-for-global-media,[],"[{'title': 22, 'chapter': 'V'}, {'title': 48, ...",United States Agency for Global Media,65391
...,...,...,...,...,...,...,...,...,...
311,Monetary Offices,,"Monetary Offices, Department of Treasury",Monetary Offices,monetary-offices,NaN,"[{'title': 31, 'chapter': 'I'}]",Department of Treasury,29020
312,Office of Financial Research,,"Office of Financial Research, Department of Tr...","Financial Research, Office of",office-of-financial-research,NaN,"[{'title': 12, 'chapter': 'XVI'}]",Department of Treasury,6039
313,Office of Foreign Assets Control,OFAC,"Office of Foreign Assets Control, Department o...","Foreign Assets Control, Office of",foreign-assets-control-office,NaN,"[{'title': 31, 'chapter': 'V'}]",Department of Treasury,558973
314,Office of Investment Security,IIO,"Office of Investment Security, Department of T...","Investment Security, Office of",investment-security-office,NaN,"[{'title': 31, 'chapter': 'VIII'}]",Department of Treasury,87539


In [40]:
all_df.groupby('parent_agency')['word_count'].sum().sort_values()

parent_agency
Federal Procurement Regulations System                    6
Appalachian Regional Commission                          85
Federal Permitting Improvement Steering Council         138
East-West Foreign Trade Board                           184
Oklahoma City National Memorial Trust                   534
                                                     ...   
Department of Transportation                        6441266
Department of Health and Human Services             7482068
Department of Agriculture                           8428528
Department of Treasury                             15480291
Environmental Protection Agency                    16715698
Name: word_count, Length: 153, dtype: int64